In [ ]:
from langgraph.graph import StateGraph, START, END
from dotenv import load_dotenv
from typing import TypedDict
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnableBranch, RunnableLambda
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel,Field
from typing import Literal

In [9]:
load_dotenv()
llm = HuggingFaceEndpoint(
    repo_id='mistralai/Mistral-7B-Instruct-v0.3',
    task='chat-completion',
    max_new_tokens=50
)

model = ChatHuggingFace(llm = llm)

In [3]:
# create a state
class LLMState(TypedDict):
    question: str
    answer: str

In [4]:
def llm_qa(state:LLMState ) -> LLMState:
    # extract the question from state
    question = state['question']
    
    # form a prompt
    prompt = f'Answer the following question {question}'
    
    # ask that question to the LLM
    answer = model.invoke(prompt).content
    
    # update the answer in the state
    state['answer'] = answer
    
    return state

In [7]:

# create our graph
graph = StateGraph(LLMState)

# add nodes
graph.add_node('llm_qa',llm_qa)

# add edges
graph.add_edge(START,'llm_qa')
graph.add_edge('llm_qa',END)

# compile
workflow = graph.compile()

In [ ]:
# execute 
intial_state = {'question':'How far is moon from the earth?'}
final_state = workflow.invoke(intial_state)
print(final_state['answer'])